In [1]:
import numpy as np
import open3d as o3d  # open3d 用于设置动画轨迹，以及保存每一帧图片
import cv2  # opencv 用于拼凑图片编辑，以及制作视频
import h5py
import os
import torch
from copy import deepcopy
import shutil
from PIL import Image, ImageDraw, ImageFont
from glob import glob

In [2]:
def create_sphere_at_xyz(xyz, colors=None, radius=0.006):
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=radius, resolution=4)
    #sphere.compute_vertex_normals()
    if colors is None:
        sphere.paint_uniform_color([0xee/0xff, 0x5c/0xff, 0x40/0xff])
    else:
        sphere.paint_uniform_color(colors)
    sphere = sphere.translate(xyz)
    return sphere

def create_point_mesh(point_cloud, colors=None, radius=0.006):
    mesh = []
    for i in range(point_cloud.shape[0]):
        mesh.append(create_sphere_at_xyz(point_cloud[i], colors=colors, radius=radius))
    
    obj = mesh[0]
    for i in range(1,len(mesh)):
        obj += mesh[i]
    return obj

In [3]:
target_point_name = 'm355'
points = np.loadtxt(os.path.join('points', '%s.xyz' % target_point_name))
obj = create_point_mesh(points)

In [ ]:
obj = o3d.io.read_triangle_mesh(os.path.join('points', '%s.obj' % target_point_name))

# 第一步 设置动画，保存路径
- Ctrl + A 添加关键帧
- Ctrl + S 保存运动轨迹文件

In [15]:
o3d.visualization.draw_geometries_with_custom_animation([obj], width=800, height=800)

设置并保存了路径，生成了.json文件
接下来，Ctrl+R运行动画，并且会截取每一帧图像

In [5]:
o3d.visualization.draw_geometries_with_custom_animation([obj], width=800, height=800, optional_view_trajectory_json_file='ViewTrajectory_2021-11-24-10-18-14.json')

# 第二步 绘制图像
为每张图像增加一些文字

In [68]:
image_paths = glob(os.path.join(target_point_name, '*.png'))
text_str = '<your_text_content>'
for image_path in image_paths:
    image = cv2.imread(image_path)
    img_PIL = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    font = ImageFont.truetype('./Roman.ttf', 40) # Consolas.ttf
    fillColor = (0, 0, 0)
    position = (360, 760)
    draw = ImageDraw.Draw(img_PIL)
    draw.text(position, text_str, font=font, fill=fillColor)
    img_OpenCV = cv2.cvtColor(np.asarray(img_PIL),cv2.COLOR_RGB2BGR)
    cv2.imwrite(image_path, img_OpenCV)

# 第三步 生成视频
将生成的图像帧绘制成视频

In [69]:
writer = cv2.VideoWriter('./demo.avi', cv2.VideoWriter_fourcc(*'DIVX'), 20.0, (3200, 3200))  # video writer

In [70]:
# cnt = len(glob(os.path.join('image', '*.png')))
cnt = 121

for i in range(1, cnt+1):
    image_basename = 'image_%06d.png' % i
    
    m355_input = cv2.imread(os.path.join('m355_input', image_basename))
    m355_pugan = cv2.imread(os.path.join('m355_pugan', image_basename))
    m355_dispu = cv2.imread(os.path.join('m355_dispu', image_basename))
    m355_ours = cv2.imread(os.path.join('m355_ours', image_basename))
    m355 = np.concatenate((m355_input, m355_pugan, m355_dispu, m355_ours), axis=1)  # 将多张图像拼在一起
    
    image = m355
    writer.write(image)
writer.release()